<a href="https://colab.research.google.com/github/sadansabo/AI-AGENTS/blob/main/Recycle_Bot_AI_Sustainability_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 🟢 AI Eco Agent (Small Model)
- TF-IDF + LogisticRegression text classifier
- Predicts: Recycle / Compost / Trash
- Single Gradio page with confidence + tips

In [10]:


!pip install -q scikit-learn gradio

import gradio as gr
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re


# Labels: Recycle, Compost, Trash
data = {
    "item": [
        # Recycle — Paper/Cardboard
        "newspaper","magazine","office paper","mixed paper","paper bag","envelope","junk mail",
        "cardboard","corrugated cardboard","shoe box","cereal box","paper packaging",
        # Recycle — Plastics (common rigid)
        "plastic bottle","water bottle","soda bottle","milk jug","detergent bottle","shampoo bottle",
        "yogurt cup","plastic container","takeout container (plastic)","clear clamshell",
        # Recycle — Metals
        "aluminum can","soda can","beer can","tin can","steel can","food can","metal lid","clean foil",
        # Recycle — Glass
        "glass bottle","wine bottle","beer bottle","glass jar","sauce jar",
        # Recycle — Other
        "newspaper bundle","paper carton","juice carton (clean)","tetra pak (clean)",
        # Compost — Food scraps & yard
        "banana peel","apple core","orange peel","vegetable scraps","carrot peels","onion skins",
        "potato skins","egg shell","coffee grounds","used coffee filter","tea bag (paper)","bread",
        "stale bread","grape stems","corn husks","leaf litter","leaves","grass clippings","twigs",
        "plant trimmings","flowers","herb stems",
        # Compost — paper/natural
        "paper towel (soiled)","paper napkin (soiled)","uncoated paper plate (soiled)","pizza box (greasy lid)",
        # Trash — soft plastics/contaminants/others
        "plastic bag","grocery bag","cling film","plastic wrap","bubble wrap","straw","plastic straw",
        "chip bag","snack wrapper","candy wrapper","foil balloon","foam tray","styrofoam","foam cup",
        "takeout container (styrofoam)","cd case","toothbrush","razor","diaper","sanitary pad",
        "wet wipes","paper towel (clean)","paper napkin (clean)","tissue (clean)","broken glass",
        "mirror","ceramic plate","light bulb","incandescent bulb","broken mirror","window glass",
        "paint can (with paint)","aerosol can (not empty)","battery","lithium battery","power bank",
        "electronics","phone","laptop","charger","cable","toys (mixed material)","shoes","old clothes",
        # Edge cases (often Recycle but with caveats)
        "foil (dirty)","greasy pizza box","greasy pizza box bottom","paper cup (plastic lined)","coffee cup (disposable)",
        "tetra pak (dirty)","juice carton (dirty)","yogurt cup (dirty)","oily plastic container","oily foil",
        # Compost edge cases (local rules vary)
        "rice","pasta","small bones","meat scraps","fish scraps","dairy"
    ],
    "category": [
        # Recycle — Paper/Cardboard (12)
        "Recycle","Recycle","Recycle","Recycle","Recycle","Recycle","Recycle",
        "Recycle","Recycle","Recycle","Recycle","Recycle",
        # Recycle — Plastics (10)
        "Recycle","Recycle","Recycle","Recycle","Recycle","Recycle",
        "Recycle","Recycle","Recycle","Recycle",
        # Recycle — Metals (8)
        "Recycle","Recycle","Recycle","Recycle","Recycle","Recycle","Recycle","Recycle",
        # Recycle — Glass (5)
        "Recycle","Recycle","Recycle","Recycle","Recycle",
        # Recycle — Other (4)
        "Recycle","Recycle","Recycle","Recycle",
        # Compost — Food/Yard (22)
        "Compost","Compost","Compost","Compost","Compost","Compost",
        "Compost","Compost","Compost","Compost","Compost","Compost",
        "Compost","Compost","Compost","Compost","Compost","Compost","Compost",
        "Compost","Compost","Compost",
        # Compost — paper/natural (4)
        "Compost","Compost","Compost","Compost",
        # Trash (33)
        "Trash","Trash","Trash","Trash","Trash","Trash","Trash",
        "Trash","Trash","Trash","Trash","Trash","Trash","Trash",
        "Trash","Trash","Trash","Trash","Trash","Trash",
        "Trash","Trash","Trash","Trash","Trash","Trash",
        "Trash","Trash","Trash","Trash","Trash","Trash","Trash",
        "Trash","Trash","Trash","Trash","Trash","Trash","Trash","Trash",
        # Edge cases — caveats (10)
        "Trash","Compost","Compost","Trash","Trash",
        "Trash","Trash","Recycle","Trash","Trash",
        # Compost edge cases (6) — choose conservative defaults
        "Trash","Trash","Trash","Trash","Trash","Trash"
    ]
}

# Corrected the category list to match the item list length
# The items "yogurt cup (dirty)" and "oily plastic container" were missing categories.
data["category"].extend(["Trash", "Trash", "Trash"]) # Added 3 'Trash' categories to match the item list length. This was the source of the mismatch.
data["category"].extend(["Trash", "Trash"]) # Adding the two missing categories for "yogurt cup (dirty)" and "oily plastic container"


print(f"Length of 'item' list: {len(data['item'])}")
print(f"Length of 'category' list: {len(data['category'])}")

df = pd.DataFrame(data)

# -----------------------------
# 2) Light text normalization
# -----------------------------
def normalize(text: str) -> str:
    text = text.strip().lower()
    text = re.sub(r"[^a-z0-9\s()/-]+", "", text)  # keep simple chars
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["item_norm"] = df["item"].apply(normalize)

# -----------------------------
# 3) Train small offline model
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    df["item_norm"], df["category"], test_size=0.2, random_state=42, stratify=df["category"]
)

model = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ("clf", LogisticRegression(max_iter=200))
])

model.fit(X_train, y_train)

# Eval (printed in Colab logs)
y_pred = model.predict(X_test)
print("=== Validation Report ===")
print(classification_report(y_test, y_pred))

# -----------------------------
# 4) Friendly tips by class
# -----------------------------
TIPS = {
    "Recycle": "Rinse items. Keep them dry. No food residue. Check local numbers for plastics.",
    "Compost": "Great for soil! Keep meat/dairy/oil out (unless your facility accepts them).",
    "Trash": "Not suitable for recycling/compost. Look for special drop-offs if hazardous."
}

# Items that should NEVER go in curbside recycling (override to Trash warning)
SPECIAL_HAZ = set([
    "battery","lithium battery","power bank","electronics","phone","laptop","charger","cable",
    "paint can (with paint)","aerosol can (not empty)","diaper","sanitary pad","wet wipes",
    "light bulb","incandescent bulb","broken mirror","mirror","window glass","styrofoam","foam cup",
    "foam tray","takeout container (styrofoam)"
])

# -----------------------------
# 5) Inference function (UI)
# -----------------------------
def predict_category(user_text: str):
    if not user_text or not user_text.strip():
        return "Please enter an item.", "", ""
    text_norm = normalize(user_text)

    # Predict
    proba = model.predict_proba([text_norm])[0]
    classes = list(model.classes_)
    # Get top prediction + confidence
    idx = proba.argmax()
    pred = classes[idx]
    conf = float(proba[idx])

    # Special hazardous override (if exact or highly indicative)
    for haz in SPECIAL_HAZ:
        if haz in text_norm:
            pred = "Trash"
            conf = max(conf, 0.9)

    # Friendly message
    if pred == "Recycle":
        msg = f"♻️ **Recycle** — “{user_text}”"
    elif pred == "Compost":
        msg = f"🌱 **Compost** — “{user_text}”"
    else:
        msg = f"🚯 **Trash** — “{user_text}”"

    tip = TIPS[pred]
    conf_msg = f"Confidence: {conf:.2f}"

    return msg, tip, conf_msg

# -----------------------------
# 6) Gradio single-page app
# -----------------------------
with gr.Blocks(title="Offline AI Eco Agent") as demo:
    gr.Markdown("# ♻️🌱 Offline AI Eco Agent (Small Model)")
    gr.Markdown(
        "Type any household item and I’ll predict whether it belongs in **Recycle**, **Compost**, or **Trash**. "
        "Runs fully offline using a compact ML model (TF-IDF + Logistic Regression)."
    )

    inp = gr.Textbox(label="Enter an item (e.g., 'milk jug', 'banana peel', 'plastic bag')", placeholder="Try: plastic bottle, pizza box, tea bag")
    btn = gr.Button("Check")
    out_main = gr.Markdown()
    out_tip = gr.Markdown()
    out_conf = gr.Markdown()

    def on_submit(x):
        return predict_category(x)

    btn.click(on_submit, inputs=inp, outputs=[out_main, out_tip, out_conf])
    inp.submit(on_submit, inputs=inp, outputs=[out_main, out_tip, out_conf])

demo.launch(share=True)

Length of 'item' list: 125
Length of 'category' list: 127


ValueError: All arrays must be of the same length

In [11]:


!pip install gradio --quiet

import gradio as gr
import difflib



recycle_data = {
    # Paper & Cardboard
    'newspaper': '♻️ Recyclable',
    'magazine': '♻️ Recyclable',
    'office paper': '♻️ Recyclable',
    'cardboard': '♻️ Recyclable',
    'paper bag': '♻️ Recyclable',

    # Plastics
    'plastic bottle': '♻️ Recyclable',
    'water bottle': '♻️ Recyclable',
    'milk jug': '♻️ Recyclable',
    'detergent bottle': '♻️ Recyclable',
    'yogurt cup': '♻️ Recyclable (check local rules)',
    'plastic container': '♻️ Recyclable (check number on bottom)',
    'plastic bag': '🚫 Not recyclable curbside (store drop-off only)',
    'cling film': '🚫 Not recyclable',
    'straw': '🚫 Not recyclable',

    # Metals
    'aluminum can': '♻️ Recyclable',
    'tin can': '♻️ Recyclable',
    'steel can': '♻️ Recyclable',
    'foil': '♻️ Recyclable if clean',
    'metal lid': '♻️ Recyclable',

    # Glass
    'glass bottle': '♻️ Recyclable',
    'glass jar': '♻️ Recyclable',
    'broken glass': '🚫 Not recyclable curbside',

    # Hazardous / Special
    'batteries': '⚠️ Hazardous waste – recycle at special collection points',
    'electronics': '⚠️ E-waste – take to e-waste recycling centers',
    'light bulb': '⚠️ Recyclable at special facilities',
    'paint can': '⚠️ Special disposal required',
    'styrofoam': '🚫 Not recyclable curbside'
}

compost_data = {
    # Fruit & Veg Waste
    'banana peel': '🌱 Compostable',
    'apple core': '🌱 Compostable',
    'orange peel': '🌱 Compostable',
    'vegetable scraps': '🌱 Compostable',
    'grape stems': '🌱 Compostable',

    # Garden Waste
    'grass clippings': '🌱 Compostable',
    'leaves': '🌱 Compostable',
    'twigs': '🌱 Compostable',
    'flowers': '🌱 Compostable',

    # Food Waste
    'coffee grounds': '🌱 Compostable',
    'tea bag': '🌱 Compostable',
    'egg shell': '🌱 Compostable',
    'bread': '🌱 Compostable in moderation',
    'rice': '🚫 Not ideal (can attract pests)',

    # Not Compostable
    'meat': '🚫 Not compostable (attracts pests)',
    'fish': '🚫 Not compostable (odor & pests)',
    'dairy': '🚫 Not compostable (odor issues)',
    'oil': '🚫 Not compostable',
    'plastic': '🚫 Not compostable',
    'metal': '🚫 Not compostable',
    'glass': '🚫 Not compostable'
}


def eco_tools(tool, item):
    item = item.lower().strip()


    data = recycle_data if tool == 'Recycle Checker' else compost_data


    if item in data:
        return data[item]


    close_matches = difflib.get_close_matches(item, data.keys(), n=1, cutoff=0.6)
    if close_matches:
        match = close_matches[0]
        return f"{data[match]} (matched: {match})"


    return "🤔 Not sure – check your local recycling or composting guidelines."


demo = gr.Interface(
    fn=eco_tools,
    inputs=[
        gr.Dropdown(['Recycle Checker', 'Compost Checker'], label='Choose Tool'),
        gr.Textbox(label='Enter an item')
    ],
    outputs="text",
    title="♻️🌱 Eco Tools Bot",
    description="Check if an item is recyclable or compostable – now with fuzzy matching & emoji fun!"
)


demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f4f225f457999a7e09.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
# Install dependencies
!pip install gradio transformers --quiet

import gradio as gr
from transformers import pipeline

# Load Hugging Face model (online, accurate)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 📘 Expanded knowledge base
knowledge_base = {
    "recyclable": [
        "plastic bottle", "glass bottle", "aluminum can", "cardboard box",
        "paper", "magazine", "newspaper", "tin can", "steel can", "carton",
        "milk jug", "plastic container", "glass jar"
    ],
    "not recyclable": [
        "plastic bag", "styrofoam", "pizza box (greasy)", "ceramics",
        "light bulb", "mirror", "battery", "electronics", "diaper",
        "food waste", "napkin", "tissue", "clothes"
    ],
    "maybe": [
        "coffee cup", "toothpaste tube", "plastic straw", "chip bag",
        "blister pack", "aerosol can", "plastic wrap"
    ]
}

# Function: check dataset first, then AI model
def check_knowledge_base(item):
    item = item.lower()
    for label, items in knowledge_base.items():
        if any(keyword in item for keyword in items):
            if label == "recyclable":
                return "♻️ Recyclable ✅"
            elif label == "not recyclable":
                return "🚫 Not recyclable ❌"
            else:
                return "🤔 Not sure – check local recycling guidelines."
    return None

def ask_recycle_bot(item):
    # Step 1: dataset check
    kb_answer = check_knowledge_base(item)
    if kb_answer:
        return kb_answer

    # Step 2: AI fallback
    labels = ["♻️ Recyclable ✅", "🚫 Not recyclable ❌", "🤔 Not sure – check local recycling guidelines."]
    result = classifier(item, candidate_labels=labels)
    return result["labels"][0]

# 🌱 Build Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## ♻️ Recycle Bot – Can I Recycle This?")
    gr.Markdown("Enter any item and I’ll tell you if it’s recyclable, not recyclable, or maybe (depends on your city).")

    with gr.Row():
        item_input = gr.Textbox(label="Enter an item", placeholder="e.g., plastic bottle")
        output = gr.Textbox(label="Recycle Bot's Answer")

    submit_btn = gr.Button("Check Item")
    submit_btn.click(fn=ask_recycle_bot, inputs=item_input, outputs=output)

# 🚀 Launch app
demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c6b5f0f55fb7c05fe2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
import gradio as gr
from transformers import pipeline

# Load Hugging Face model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Knowledge base
knowledge_base = {
    "recyclable": [
        "plastic bottle", "glass bottle", "aluminum can", "cardboard box",
        "paper", "magazine", "newspaper", "tin can", "steel can", "carton",
        "milk jug", "plastic container", "glass jar"
    ],
    "not recyclable": [
        "plastic bag", "styrofoam", "pizza box (greasy)", "ceramics",
        "light bulb", "mirror", "battery", "electronics", "diaper",
        "food waste", "napkin", "tissue", "clothes"
    ],
    "maybe": [
        "coffee cup", "toothpaste tube", "plastic straw", "chip bag",
        "blister pack", "aerosol can", "plastic wrap"
    ]
}

# Dataset check
def check_knowledge_base(item):
    item = item.lower()
    for label, items in knowledge_base.items():
        if any(keyword in item for keyword in items):
            if label == "recyclable":
                return "♻️ Recyclable ✅"
            elif label == "not recyclable":
                return "🚫 Not recyclable ❌"
            else:
                return "🤔 Not sure – check local recycling guidelines."
    return None

# Main function
def ask_recycle_bot(item):
    kb_answer = check_knowledge_base(item)
    if kb_answer:
        return kb_answer

    labels = ["♻️ Recyclable ✅", "🚫 Not recyclable ❌", "🤔 Not sure – check local recycling guidelines."]
    result = classifier(item, candidate_labels=labels)
    return result["labels"][0]

# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## ♻️ Recycle Bot – Can I Recycle This?")
    gr.Markdown("Enter any item and I’ll tell you if it’s recyclable, not recyclable, or maybe (depends on your city).")

    with gr.Row():
        item_input = gr.Textbox(label="Enter an item", placeholder="e.g., plastic bottle")
        output = gr.Textbox(label="Recycle Bot's Answer")

    submit_btn = gr.Button("Check Item")
    submit_btn.click(fn=ask_recycle_bot, inputs=item_input, outputs=output)

demo.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd8211db470b089bf3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [27]:
# ♻️ Recycle Bot

A smart AI bot that tells you whether an item is recyclable, not recyclable, or maybe.

- Uses a **knowledge base** for common items.
- Falls back to **Hugging Face AI model (facebook/bart-large-mnli)** for general classification.
- Built with **Gradio** for a simple web interface.

Try it out directly in Hugging Face Spaces 🚀


SyntaxError: invalid character '🚀' (U+1F680) (ipython-input-2312915883.py, line 9)

In [28]:
# 🚀 STEP 1: Install dependencies
!pip install transformers torch --quiet

# 🚀 STEP 2: Import libraries
from transformers import pipeline

# 🚀 STEP 3: Load a larger pre-trained model
# We'll use a zero-shot classifier so we don’t need to train from scratch
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🚀 STEP 4: Define labels (categories for recycling)
labels = ["Recyclable", "Not Recyclable", "Check Local Guidelines"]

# 🚀 STEP 5: Function to classify items
def recycle_bot(item):
    result = classifier(item, candidate_labels=labels)
    return f"♻️ Item: {item}\n✅ Classification: {result['labels'][0]} (confidence: {result['scores'][0]:.2f})"

# 🚀 STEP 6: Interactive loop
print("♻️ Welcome to Recycle Bot (AI Powered)!")
print("Type an item to check (or 'quit' to stop)")

while True:
    item = input("Enter an item: ").strip().lower()
    if item == "quit":
        print("👋 Goodbye! Stay green 🌱")
        break
    print(recycle_bot(item))


Device set to use cpu


♻️ Welcome to Recycle Bot (AI Powered)!
Type an item to check (or 'quit' to stop)
Enter an item: 'quit'
♻️ Item: 'quit'
✅ Classification: Not Recyclable (confidence: 0.52)
Enter an item: quit
👋 Goodbye! Stay green 🌱


In [22]:
https://huggingface.co/spaces/sadansabo/recycle-bot


SyntaxError: invalid syntax (ipython-input-2107754534.py, line 1)

In [25]:
!git add app.py && git commit -m '/content/recycle-bot' && git push

fatal: not a git repository (or any of the parent directories): .git


In [30]:
gradio
transformers
torch


NameError: name 'gradio' is not defined

# ♻️ Recycle Bot

A smart AI bot that tells you whether an item is recyclable, not recyclable, or maybe.

- Uses a **knowledge base** for common items.
- Falls back to **Hugging Face AI model (facebook/bart-large-mnli)** for general classification.
- Built with **Gradio** for a simple web interface.

Try it out directly in Hugging Face Spaces 🚀

In [31]:
# ===============================
# ♻️ Recycle-Bot Colab Notebook
# ===============================

# Step 1: Install dependencies
!pip install gradio transformers

# Step 2: Import libraries
import gradio as gr
from transformers import pipeline

# Step 3: Load a small, offline model (DistilBERT Q&A)
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Step 4: Define knowledge base
knowledge_base = """
Recycle-Bot is an AI agent designed to promote sustainability.
It answers recycling questions like what can be recycled, how to dispose of waste,
and educates users about environmental care. Brand: GREEN TIME.

Examples:
- Plastic bottles: Rinse and recycle in plastics bin.
- Glass: Recycle in the glass bin.
- Electronics: Take to e-waste collection centers.
- Organic waste: Compost if possible.
"""

# Step 5: Chat function
def answer_question(question):
    result = qa_pipeline(question=question, context=knowledge_base)
    return result['answer']

# Step 6: Gradio Interface
demo = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(label="Ask Recycle-Bot ♻️"),
    outputs=gr.Textbox(label="Recycle-Bot Answer"),
    title="♻️ Recycle-Bot (GREEN TIME)",
    description="An AI Agent that helps you recycle and protect the environment."
)

# Step 7: Launch in Colab
demo.launch(share=True)

# ===============================
# Project Summary & Description
# ===============================
project_summary = """
# ♻️ Recycle-Bot: AI Sustainability Agent

## Project Description
Recycle-Bot (Brand: GREEN TIME) is an AI-powered chatbot that educates people about recycling and sustainability.
It answers user queries about waste disposal, recycling categories, and eco-friendly practices.

## Objectives
- Help individuals recycle correctly.
- Reduce waste contamination.
- Promote environmental awareness.

## How It Works
- Uses a lightweight NLP model (DistilBERT).
- Knowledge base is focused on recycling rules.
- Deployed with Gradio in Colab & Hugging Face Spaces.

## Demo Use Cases
- Q: Can I recycle glass bottles?
  A: Yes, place them in the glass bin.
- Q: What do I do with food waste?
  A: Compost it if possible.

## Impact
Recycle-Bot contributes to sustainability by simplifying recycling education, encouraging responsible waste management, and aligning with SDG 13 (Climate Action).
"""

# Save summary as text file for presentation
with open("Recycle_Bot_Summary.txt", "w") as f:
    f.write(project_summary)

print("✅ Project summary saved as Recycle_Bot_Summary.txt")


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d7088c56c2e390dcf0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Project summary saved as Recycle_Bot_Summary.txt
